## Important notes: 
 - it is necessary to use a GPU for computing peak memory consumption, since we are using CUDA library for this.
 - it is necessary to make a free account on SigOpt platform to recieve own API token to be able to run experiments.

### Install neccessary packages

In [ ]:
!pip install torch torchvision
!pip install sigopt
!pip install ipywidgets
!pip install plotly

### Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
import plotly.graph_objects as go

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import sigopt
import os
import time 

### Set up connection to SigOpt platform and using GPU

In [ ]:
os.environ["SIGOPT_API_TOKEN"] = #SIGOPT API TOKEN
os.environ["SIGOPT_PROJECT"] = "model_optimization_00000"
%reload_ext sigopt

dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(dtype, device)

### Model 

In [ ]:
class Model(nn.Module):
    def __init__(self, image_width, image_channels, num_classes, params):
        super().__init__()
        self.conv1 = nn.Conv2d(image_channels, params["conv1_out_channels"], params["conv1_kernel_size"], padding="same")
        self.conv2 = nn.Conv2d(params["conv1_out_channels"], params["conv2_out_channels"], params["conv2_kernel_size"], padding="same")
        self.conv3 = nn.Conv2d(params["conv2_out_channels"], params["conv3_out_channels"], params["conv3_kernel_size"], padding="same")
        self.pool = nn.MaxPool2d(params["pool_kernel_size"])
        self.batchnorm1 = nn.BatchNorm2d(params["conv1_out_channels"])
        self.batchnorm2 = nn.BatchNorm2d(params["conv2_out_channels"])
        self.batchnorm3 = nn.BatchNorm2d(params["conv3_out_channels"])
        dim_after_two_pools = int(image_width / (params["pool_kernel_size"] ** 2))
        self.fc1 = nn.Linear(params["conv3_out_channels"] * dim_after_two_pools ** 2, params["fc1_out_features"])
        self.fc2 = nn.Linear(params["fc1_out_features"], params["fc2_out_features"])
        self.fc3 = nn.Linear(params["fc2_out_features"], num_classes)
        self.dropout = nn.Dropout(params["dropout_probability"])

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.relu(self.conv2(x))
        x = self.pool(self.batchnorm2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(self.batchnorm3(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Loss and optimizer 

In [ ]:
def loss_and_optimizer(model, learning_rate):
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return loss, optimizer

### Train function

In [ ]:
def train(dataloader, model, criterion, optimizer, epochs):
    loss_array = []
    accuracy_array = []
    
    for t in range(epochs):
        print(f"\nEpoch {t+1}\n-------------------------------")
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        train_loss, correct = 0, 0
        
        model.train()     
        for batch, (X, y) in enumerate(dataloader):
            X = X.to(dtype=dtype, device=device)
            y = y.to(device=device)
            
            # Compute prediction error
            pred = model(X)
            loss = criterion(pred, y)
            
            # Update 
            train_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 1000 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
                
        train_loss /= num_batches
        loss_array.append(train_loss)       
        correct /= size
        accuracy_array.append(100*correct)
        print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

### Test function

In [ ]:
def test(dataloader, model, criterion):
    # evaluate mode
    model.eval()
    
    """Compute the different metrics"""
    
    metrics = {}
    
    # accuracy and loss
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss, correct = 0, 0
    
    # inference time
    total_time = 0
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    
    # gpu warm-up
    dummy_input = torch.randn(8, 3, 32, 32, dtype=dtype).to(device) # for CIFAR 10 -> batch_size=8, image_channels=3, image_width=32
    for _ in range(10):
        _ = model(dummy_input)
        
    # reset peak memory stats - just compute the peak memory for inference
    torch.cuda.reset_peak_memory_stats()
    
    # inference
    with torch.no_grad():
        for X, y in dataloader:        
            X = X.to(dtype=dtype, device=device)
            y = y.to(device=device)  
            
            # record time
            starter.record()
            pred = model(X) 
            ender.record()
      
            # wait for gpu sync
            torch.cuda.synchronize()
            
            curr_time = starter.elapsed_time(ender)
            total_time += curr_time  
            loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()            
   
    #peak memory - in mB per image 
    stats = torch.cuda.memory_stats(device=device)
    peak_memory = stats["allocated_bytes.all.peak"] / (2 ** 20)
    peak_memory /= size
    
    #storage memory - in mB
    torch.save(model.state_dict(), "tmp2.pt")
    size_mb = os.path.getsize("tmp2.pt")/1e6
    os.remove('tmp2.pt')
    storage_memory = np.round(size_mb, 2)

    # accuracy and loss
    correct /= size  
    
    # inference time - in seconds per image
    inference_time = total_time / size
    
    metrics['accuracy'] = correct
    metrics['inference_time'] = inference_time
    metrics['peak_memory'] = peak_memory
    metrics['storage_memory'] = storage_memory
    
    print(f"Test accuracy: {(100*correct):>0.1f}% \n")
    
    return metrics

### Radar plot function

In [ ]:
class ComplexRadar():
    def __init__(self, fig, variables, ranges, n_ordinate_levels=6):
        angles = np.arange(0, 360, 360./len(variables))

        axes = [fig.add_axes([0.1,0.1,0.9,0.9], polar=True,
            label = "axes{}".format(i)) 
            for i in range(len(variables))]

        l, text = axes[0].set_thetagrids(angles, labels=variables)

        [[txt.set_fontweight('bold'),
              txt.set_fontsize(12),
              txt.set_position((0,-0.2))] for txt in text]

        for ax in axes[1:]:
            ax.patch.set_visible(False)
            ax.grid("off")
            ax.xaxis.set_visible(False)

        for i, ax in enumerate(axes):
            grid = np.linspace(*ranges[i], num=n_ordinate_levels)
            gridlabel = ["{}".format(round(x,2)) for x in grid]

            gridlabel[0] = "" # clean up origin
            ax.set_rgrids(grid, labels=gridlabel,angle=angles[i])

            ax.set_ylim(*ranges[i])
        
        # variables for plotting
        self.angle = np.deg2rad(np.r_[angles, angles[0]])
        self.ranges = ranges
        self.ax = axes[0]

    def plot(self, data, *args, **kw):
        sdata = self.scale_data(data, self.ranges)
        self.ax.plot(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def fill(self, data, *args, **kw):
        sdata = self.scale_data(data, self.ranges)
        self.ax.fill(self.angle, np.r_[sdata, sdata[0]], *args, **kw)

    def scale_data(self, data, ranges):
        """scales data[1:] to ranges[0]"""
        for d, (y1, y2) in zip(data[1:], ranges[1:]):
            assert (y1 <= d <= y2) or (y2 <= d <= y1)
        x1, x2 = ranges[0]
        d = data[0]
        sdata = [d]
        for d, (y1, y2) in zip(data[1:], ranges[1:]):
            if y1 > y2:
                d = _invert(d, (y1, y2))
                y1, y2 = y2, y1
            sdata.append((d-y1) / (y2-y1) * (x2 - x1) + x1)
        return sdata

### Optimize with SigOpt

In [ ]:
def train_and_track_model(trainset, testset, testbatchsize, image_width, image_channels, classes, run):

    """
    Creates a function which trains a model while tracking artifacts
    """
  
    run.log_model('CNN with 3 convolutional, 2 pools and 2 fully connected layers')
    run.log_dataset(name='CIFAR_10')

    run.params.setdefault("batchsize", 32)
    run.params.setdefault("learning_rate", 0.001)
    run.params.setdefault("epochs", 5)
    run.params.setdefault("conv1_out_channels", 32)
    run.params.setdefault("conv1_kernel_size", 5)
    run.params.setdefault("pool_kernel_size", 2)
    run.params.setdefault("conv2_out_channels", 64)
    run.params.setdefault("conv2_kernel_size", 5)
    run.params.setdefault("conv3_out_channels", 128)
    run.params.setdefault("conv3_kernel_size", 5)
    run.params.setdefault("fc1_out_features", 512)
    run.params.setdefault("fc2_out_features", 256)
    run.params.setdefault("dropout_probability", 0.2)

    # construct training data loader
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=run.params.batchsize, shuffle="True")
    
    # construct testing data loader
    test_loader = torch.utils.data.DataLoader(testset, batch_size=testbatchsize, shuffle="True")

    # initialize the model
    model = Model(image_width, image_channels, len(classes), run.params)
    
    # change dtype and device of the model
    model.to(dtype=dtype, device=device)

    # define loss and optimizer
    loss, optimizer = loss_and_optimizer(model, run.params.learning_rate)
    
    # train network
    train(train_loader, model, loss, optimizer, run.params.epochs)

    # test network, returns values of metrics
    metrics = test(test_loader, model, loss)
    
    # log metrics
    for name, value in metrics.items():
        run.log_metric(name, value)

### Download CIFAR10 datasets

In [ ]:
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

classes = testset.classes
channels, width, height = trainset[0][0].shape

print(channels, width, height, classes)

## Experiment 1: Optimizing accuracy while only keeping track of the other metrics - with test batch size 10

In [ ]:
experiment_acc10 = sigopt.create_experiment(
  name="AutoML_acc_test_batch_size_10",
  type="offline",
  parameters=[
      dict(name="batchsize", type="int", bounds=dict(min=8, max=64)),
      dict(name="learning_rate", type="double", bounds=dict(min=1.0e-5, max=0.5), transformation="log"),
      dict(name="epochs", type="int", bounds=dict(min=3, max=20)),
      dict(name="conv1_out_channels", type="int", bounds=dict(min=16, max=64)),
      dict(name="conv1_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="pool_kernel_size", type="int", bounds=dict(min=2, max=3)),
      dict(name="conv2_out_channels", type="int", bounds=dict(min=32, max=128)),
      dict(name="conv2_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="conv3_out_channels", type="int", bounds=dict(min=64, max=256)),
      dict(name="conv3_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="fc1_out_features", type="int", bounds=dict(min=256, max=1024)),
      dict(name="fc2_out_features", type="int", bounds=dict(min=64, max=512)),
      dict(name="dropout_probability", type="double", bounds=dict(min=0.01, max=0.5))
  ],
  metrics=[
      dict(name="accuracy", objective="maximize", strategy="optimize"),
      dict(name="inference_time", objective="minimize", strategy="store"),
      dict(name="peak_memory", objective="minimize", strategy="store"),
      dict(name="storage_memory", objective="minimize", strategy="store")
  ],
  parallel_bandwidth=1,
  budget=100,
)

In [ ]:
for run in experiment_acc10.loop():
    with run:
        train_and_track_model(trainset, testset, 10, width, channels, classes, run)

In [ ]:
# get the best run for the experiment
best_runs_acc10 = experiment_acc10.get_best_runs()
best_runs_acc10 = list(best_runs_acc10)
for run in best_runs_acc10:
    print(run)

In [ ]:
# plot results
runs = experiment_acc10.get_runs()
runs = list(runs)
runs = runs[::-1]
length = len(runs)

accuracies = []
times = []
pm = []
sm = []

for run in runs:
    if(run.state == "completed"):
        accuracies.append(run.values["accuracy"].value)
        times.append(run.values["inference_time"].value)
        pm.append(run.values["peak_memory"].value)
        sm.append(run.values["storage_memory"].value)
        
fig, axs = plt.subplots(2, 2, figsize=(16,12), sharex=True)
axs[0, 0].plot(accuracies, '-o')
axs[0, 0].set_title('Accuracy')
axs[0, 0].set(ylabel='[%]')
axs[0, 1].plot(times, '-o')
axs[0, 1].set_title('Inference time')
axs[0, 1].set(ylabel='[seconds per image]')
axs[1, 0].plot(pm, '-o')
axs[1, 0].set_title('Peak memory consumption')
axs[1, 0].set(xlabel='Runs', ylabel='[MB per image]')
axs[1, 1].plot(sm, '-o')
axs[1, 1].set_title('Storage memory')
axs[1, 1].set(xlabel='Runs', ylabel='[MB]')
fig.tight_layout()

## Experiment 1: Optimizing accuracy while only keeping track of the other metrics - with test batch size 20

In [ ]:
experiment_acc20 = sigopt.create_experiment(
  name="AutoML_acc_test_batch_size_20",
  type="offline",
  parameters=[
      dict(name="batchsize", type="int", bounds=dict(min=8, max=64)),
      dict(name="learning_rate", type="double", bounds=dict(min=1.0e-5, max=0.5), transformation="log"),
      dict(name="epochs", type="int", bounds=dict(min=3, max=20)),
      dict(name="conv1_out_channels", type="int", bounds=dict(min=16, max=64)),
      dict(name="conv1_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="pool_kernel_size", type="int", bounds=dict(min=2, max=3)),
      dict(name="conv2_out_channels", type="int", bounds=dict(min=32, max=128)),
      dict(name="conv2_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="conv3_out_channels", type="int", bounds=dict(min=64, max=256)),
      dict(name="conv3_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="fc1_out_features", type="int", bounds=dict(min=256, max=1024)),
      dict(name="fc2_out_features", type="int", bounds=dict(min=64, max=512)),
      dict(name="dropout_probability", type="double", bounds=dict(min=0.01, max=0.5))
  ],
  metrics=[
      dict(name="accuracy", objective="maximize", strategy="optimize"),
      dict(name="inference_time", objective="minimize", strategy="store"),
      dict(name="peak_memory", objective="minimize", strategy="store"),
      dict(name="storage_memory", objective="minimize", strategy="store")
  ],
  parallel_bandwidth=1,
  budget=100,
)

In [ ]:
for run in experiment_acc20.loop():
    with run:
        train_and_track_model(trainset, testset, 20, width, channels, classes, run)

In [ ]:
# get the best run for the experiment
best_runs_acc20 = experiment_acc20.get_best_runs()
best_runs_acc20 = list(best_runs_acc20)
for run in best_runs_acc20:
    print(run)

In [ ]:
# plot results
runs = experiment_acc20.get_runs()
runs = list(runs)
runs = runs[::-1]
length = len(runs)

accuracies = []
times = []
pm = []
sm = []

for run in runs:
    if(run.state == "completed"):
        accuracies.append(run.values["accuracy"].value)
        times.append(run.values["inference_time"].value)
        pm.append(run.values["peak_memory"].value)
        sm.append(run.values["storage_memory"].value)
        
fig, axs = plt.subplots(2, 2, figsize=(16,12), sharex=True)
axs[0, 0].plot(accuracies, '-o')
axs[0, 0].set_title('Accuracy')
axs[0, 0].set(ylabel='[%]')
axs[0, 1].plot(times, '-o')
axs[0, 1].set_title('Inference time')
axs[0, 1].set(ylabel='[seconds per image]')
axs[1, 0].plot(pm, '-o')
axs[1, 0].set_title('Peak memory consumption')
axs[1, 0].set(xlabel='Runs', ylabel='[MB per image]')
axs[1, 1].plot(sm, '-o')
axs[1, 1].set_title('Storage memory')
axs[1, 1].set(xlabel='Runs', ylabel='[MB]')
fig.tight_layout()

## Experiment 2: Constraint Active Search - with test batch size 10

In [ ]:
experiment_cas10 = sigopt.create_experiment(
  name="AutoML_cas_test_batch_size_10",
  type="offline",
  parameters=[
      dict(name="batchsize", type="int", bounds=dict(min=8, max=64)),
      dict(name="learning_rate", type="double", bounds=dict(min=1.0e-5, max=0.5), transformation="log"),
      dict(name="epochs", type="int", bounds=dict(min=3, max=20)),
      dict(name="conv1_out_channels", type="int", bounds=dict(min=16, max=64)),
      dict(name="conv1_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="pool_kernel_size", type="int", bounds=dict(min=2, max=3)),
      dict(name="conv2_out_channels", type="int", bounds=dict(min=32, max=128)),
      dict(name="conv2_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="conv3_out_channels", type="int", bounds=dict(min=64, max=256)),
      dict(name="conv3_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="fc1_out_features", type="int", bounds=dict(min=256, max=1024)),
      dict(name="fc2_out_features", type="int", bounds=dict(min=64, max=512)),
      dict(name="dropout_probability", type="double", bounds=dict(min=0.01, max=0.5))
  ],
  metrics=[
      dict(name="accuracy", objective="maximize", strategy="constraint", threshold=0.75),
      dict(name="inference_time", objective="minimize", strategy="constraint", threshold=0.3),
      dict(name="peak_memory", objective="minimize", strategy="constraint", threshold=0.01),
      dict(name="storage_memory", objective="minimize", strategy="constraint", threshold=10)
  ],
  parallel_bandwidth=1,
  budget=100,
)

In [ ]:
for run in experiment_cas10.loop():
    with run:
        train_and_track_model(trainset, testset, 10, width, channels, classes, run)

In [ ]:
# get the best run for the experiment
best_runs_cas10 = experiment_cas10.get_best_runs()
best_runs_cas10 = list(best_runs_cas10)
for run in best_runs_cas10:
    print(run)

In [ ]:
# how many best runs - runs that satisfy constraints - is there
print(len(best_runs_cas10))

In [ ]:
# plot results
runs = experiment_cas10.get_runs()
runs = list(runs)
runs = runs[::-1]
length = len(runs)

accuracies = []
times = []
pm = []
sm = []

for run in runs:
    if(run.state == "completed"):
        accuracies.append(run.values["accuracy"].value)
        times.append(run.values["inference_time"].value)
        pm.append(run.values["peak_memory"].value)
        sm.append(run.values["storage_memory"].value)
        
fig, axs = plt.subplots(2, 2, figsize=(16,12), sharex=True)
axs[0, 0].plot(accuracies, '-o')
axs[0, 0].set_title('Accuracy')
axs[0, 0].set(ylabel='[%]')
axs[0, 1].plot(times, '-o')
axs[0, 1].set_title('Inference time')
axs[0, 1].set(ylabel='[seconds per image]')
axs[1, 0].plot(pm, '-o')
axs[1, 0].set_title('Peak memory consumption')
axs[1, 0].set(xlabel='Runs', ylabel='[MB per image]')
axs[1, 1].plot(sm, '-o')
axs[1, 1].set_title('Storage memory')
axs[1, 1].set(xlabel='Runs', ylabel='[MB]')
fig.tight_layout()

## Experiment 2: Constraint Active Search - with test batch size 20

In [ ]:
experiment_cas20 = sigopt.create_experiment(
  name="AutoML_cas_test_batch_size_20",
  type="offline",
  parameters=[
      dict(name="batchsize", type="int", bounds=dict(min=8, max=64)),
      dict(name="learning_rate", type="double", bounds=dict(min=1.0e-5, max=0.5), transformation="log"),
      dict(name="epochs", type="int", bounds=dict(min=3, max=20)),
      dict(name="conv1_out_channels", type="int", bounds=dict(min=16, max=64)),
      dict(name="conv1_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="pool_kernel_size", type="int", bounds=dict(min=2, max=3)),
      dict(name="conv2_out_channels", type="int", bounds=dict(min=32, max=128)),
      dict(name="conv2_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="conv3_out_channels", type="int", bounds=dict(min=64, max=256)),
      dict(name="conv3_kernel_size", type="int", grid=[3,5,7,9]),
      dict(name="fc1_out_features", type="int", bounds=dict(min=256, max=1024)),
      dict(name="fc2_out_features", type="int", bounds=dict(min=64, max=512)),
      dict(name="dropout_probability", type="double", bounds=dict(min=0.01, max=0.5))
  ],
  metrics=[
      dict(name="accuracy", objective="maximize", strategy="constraint", threshold=0.75),
      dict(name="inference_time", objective="minimize", strategy="constraint", threshold=0.3),
      dict(name="peak_memory", objective="minimize", strategy="constraint", threshold=0.01),
      dict(name="storage_memory", objective="minimize", strategy="constraint", threshold=10)
  ],
  parallel_bandwidth=1,
  budget=100,
)

In [ ]:
for run in experiment_cas20.loop():
    with run:
        train_and_track_model(trainset, testset, 20, width, channels, classes, run)

In [ ]:
# get the best run for the experiment
best_runs_cas20 = experiment_cas20.get_best_runs()
best_runs_cas20 = list(best_runs_cas20)
for run in best_runs_cas20:
    print(run)

In [ ]:
# how many best runs - runs that satisfy constraints - is there
print(len(best_runs_cas20))

In [ ]:
# plot results
runs = experiment_cas20.get_runs()
runs = list(runs)
runs = runs[::-1]
length = len(runs)

accuracies = []
times = []
pm = []
sm = []

for run in runs:
    if(run.state == "completed"):
        accuracies.append(run.values["accuracy"].value)
        times.append(run.values["inference_time"].value)
        pm.append(run.values["peak_memory"].value)
        sm.append(run.values["storage_memory"].value)
        
fig, axs = plt.subplots(2, 2, figsize=(16,12), sharex=True)
axs[0, 0].plot(accuracies, '-o')
axs[0, 0].set_title('Accuracy')
axs[0, 0].set(ylabel='[%]')
axs[0, 1].plot(times, '-o')
axs[0, 1].set_title('Inference time')
axs[0, 1].set(ylabel='[seconds per image]')
axs[1, 0].plot(pm, '-o')
axs[1, 0].set_title('Peak memory consumption')
axs[1, 0].set(xlabel='Runs', ylabel='[MB per image]')
axs[1, 1].plot(sm, '-o')
axs[1, 1].set_title('Storage memory')
axs[1, 1].set(xlabel='Runs', ylabel='[MB]')
fig.tight_layout()

## Radar plot for comparison of best results from Experiment 1 and Experiment 2

Note: It is necessary to find IDs of runs with the best results from current runs of experiments. The easiest way to get those is to access SigOpt platform -> current project -> experiment -> "History", and by using the sorting option find the run ID with highest accuracy for Experiment 1, and IDs of runs with highest accuracy, lowest inference time, lowest peak memory consumption, and lowest storage memory for Experiment 2. 

In [ ]:
# test batch size 10
id_baseline_run = #Run_ID
id_CAS_acc_run = #Run_ID
id_CAS_time_run = #Run_ID
id_CAS_pm_run = #Run_ID
id_CAS_sm_run = #Run_ID

# test batch size 20
#id_baseline_run = #Run_ID
#id_CAS_acc_run = #Run_ID
#id_CAS_time_run = #Run_ID
#id_CAS_pm_run = #Run_ID
#id_CAS_sm_run = #Run_ID

run_baseline = sigopt.get_run(id_baseline_run).values
run_CAS_acc = sigopt.get_run(id_CAS_acc_run).values
run_CAS_time = sigopt.get_run(id_CAS_time_run).values
run_CAS_pm = sigopt.get_run(id_CAS_pm_run).values
run_CAS_sm = sigopt.get_run(id_CAS_sm_run).values

metrics_baseline = []
metrics_CAS_acc = []
metrics_CAS_time = []
metrics_CAS_pm = []
metrics_CAS_sm = []

variables = ['accuracy', 'inference_time', 'peak_memory', 'storage_memory']
ranges = [(0, 1), (0, 0.4), (0, 0.06), (1, 60)] 

for k in variables:
    metrics_baseline.append(run_baseline[k].value)
    metrics_CAS_acc.append(run_CAS_acc[k].value)
    metrics_CAS_time.append(run_CAS_time[k].value)
    metrics_CAS_pm.append(run_CAS_pm[k].value)
    metrics_CAS_sm.append(run_CAS_sm[k].value)

# plotting
fig1 = plt.figure(figsize=(6, 6))
fig1.suptitle("Test batch size 1", x = 0.55, y=1.25, fontweight = "bold")
radar = ComplexRadar(fig1, variables, ranges)

radar.plot(metrics_baseline, label="E1: Best accuracy")
radar.fill(metrics_baseline, alpha=0.2)

radar.plot(metrics_CAS_acc, label="E2: Best accuracy")
radar.fill(metrics_CAS_acc, alpha=0.2)

radar.plot(metrics_CAS_time, label="E2: Best inference time")
radar.fill(metrics_CAS_time, alpha=0.2)

radar.plot(metrics_CAS_pm, label="E2: Best peak memory")
radar.fill(metrics_CAS_pm, alpha=0.2)

radar.plot(metrics_CAS_sm, label="E2: Best storage memory")
radar.fill(metrics_CAS_sm, alpha=0.2)
    
radar.ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.20),
      fancybox=True, ncol=5, fontsize=12)
plt.show()

# Parallel Coordinate Plot

Note: these are the constraints used for the experiment

In [ ]:
constraints = {}
constraints['accuracy'] = 0.75
constraints['inference_time'] = 0.3
constraints['peak_memory'] = 100.0/10000.0
constraints['storage_memory'] = 10.0

In [ ]:
experiment_bs_10 = " insert experiment ID for batch size of 10 " # it is not nessesary that this experiment have a btch size of 10, it can be any experiment as long as all of the parameters and metrics are there. E.g., other optimization algorithms
experiment_bs_10 = sigopt.get_experiment(experiment_bs_10)

df_bs_10 = pd.DataFrame([])

for run in experiment_bs_10.get_runs():

  if df_bs_10.empty:
    
    metrics_names = [metric for metric in run.values.keys()]
    hyperparameters_names = [hyperparameter for hyperparameter in run.assignments.keys()]
    
    df_bs_10 = pd.DataFrame([], columns = metrics_names + hyperparameters_names)
  
  metrics = {}
  for m in run.values: metrics[run.values[m].name] = run.values[m].value

  metrics.update(run.assignments)

  df_bs_10 = df_bs_10.append(metrics, ignore_index=True)

df_bs_10['peak_memory'] = df_bs_10['peak_memory'] / 10000.0

hue_bs_10 = pd.DataFrame([(df_bs_10['accuracy'] > constraints['accuracy']),
                        (df_bs_10['inference_time'] < constraints['inference_time']),
                        (df_bs_10['peak_memory'] < constraints['peak_memory']),
                        (df_bs_10['storage_memory'] < constraints['storage_memory'])]).all()

hue_bs_10.name = 'Constraints'

hue_bs_10[hue_bs_10 == False] = 0
hue_bs_10[hue_bs_10 == True] = 1

df_bs_10 = pd.concat([df_bs_10, hue_bs_10], axis = 1)

In [ ]:
experiment_bs_20 = " insert experiment ID for batch size of 20 " # it is not nessesary that this experiment have a btch size of 10, it can be any experiment as long as all of the parameters and metrics are there. E.g., other optimization algorithms
experiment_bs_20 = sigopt.get_experiment(experiment_bs_20)

df_bs_20 = pd.DataFrame([])

for run in experiment_bs_20.get_runs():

  if df_bs_20.empty:
    
    metrics_names = [metric for metric in run.values.keys()]
    hyperparameters_names = [hyperparameter for hyperparameter in run.assignments.keys()]
    
    df_bs_20 = pd.DataFrame([], columns = metrics_names + hyperparameters_names)
  
  metrics = {}
  for m in run.values: metrics[run.values[m].name] = run.values[m].value

  metrics.update(run.assignments)

  df_bs_20 = df_bs_20.append(metrics, ignore_index=True)

df_bs_20['peak_memory'] = df_bs_20['peak_memory'] / 10000.0

hue_bs_20 = pd.DataFrame([(df_bs_20['accuracy'] > constraints['accuracy']),
                          (df_bs_20['inference_time'] < constraints['inference_time']),
                          (df_bs_20['peak_memory'] < constraints['peak_memory']),
                          (df_bs_20['storage_memory'] < constraints['storage_memory'])]).all()

hue_bs_20.name = 'Constraints'

hue_bs_20[hue_bs_20 == False] = 0
hue_bs_20[hue_bs_20 == True] = 1

df_bs_20 = pd.concat([df_bs_20, hue_bs_20], axis = 1)

In [ ]:
range_accuracy = [min(df_bs_10['accuracy'].min(),
                      df_bs_20['accuracy'].min()),
                  max(df_bs_10['accuracy'].max(),
                      df_bs_20['accuracy'].max())]

range_peak_memory = [min(df_bs_10['peak_memory'].min(),
                         df_bs_20['peak_memory'].min()),
                     max(df_bs_10['peak_memory'].max(),
                         df_bs_20['peak_memory'].max())]

range_storage_memory = [min(df_bs_10['storage_memory'].min(),
                            df_bs_20['storage_memory'].min()),
                        max(df_bs_10['storage_memory'].max(),
                            df_bs_20['storage_memory'].max())]

range_inference_time = [min(df_bs_10['inference_time'].min(),
                            df_bs_20['inference_time'].min()),
                        max(df_bs_10['inference_time'].max(),
                            df_bs_20['inference_time'].max())]

In [ ]:
dimensions_hyperparameters = list([
                                   dict(range = [8, 64], label = 'batchsize', values = df_bs_10['batchsize']),
                                   dict(range = [3, 9], label = 'conv1_kernel_size', values = df_bs_10['conv1_kernel_size']),
                                   dict(range = [16, 64], label = 'conv1_out_channels', values = df_bs_10['conv1_out_channels']),
                                   dict(range = [3, 9], label = 'conv2_kernel_size', values = df_bs_10['conv2_kernel_size']),
                                   dict(range = [32, 128], label = 'conv2_out_channels', values = df_bs_10['conv2_out_channels']),
                                   dict(range = [3, 9], label = 'conv3_kernel_size', values = df_bs_10['conv3_kernel_size']),
                                   dict(range = [64, 256], label = 'conv3_out_channels', values = df_bs_10['conv3_out_channels']),
                                   dict(range = [0.01, 0.5], label = 'dropout_probability', values = df_bs_10['dropout_probability']),
                                   dict(range = [3, 20], label = 'epochs', values = df_bs_10['epochs']),
                                   dict(range = [256, 1024], label = 'fc1_out_features', values = df_bs_10['fc1_out_features']),
                                   dict(range = [64, 512]	, label = 'fc2_out_features', values = df_bs_10['fc2_out_features']),
                                   dict(range = [0.00001, 0.5], label = 'learning_rate', values = df_bs_10['learning_rate']),
                                   dict(range = [2, 3], label = 'pool_kernel_size', values = df_bs_10['pool_kernel_size']),
                                   ])

dimensions_metrics = list([
                           dict(range = range_accuracy, label = 'accuracy', values = df_bs_10['accuracy']),
                           dict(range = range_peak_memory, label = 'peak_memory', values = df_bs_10['peak_memory']),
                           dict(range = range_storage_memory, label = 'storage_memory', values = df_bs_10['storage_memory']),
                           dict(range = range_inference_time, label = 'inference_time', values = df_bs_10['inference_time']),
                           ])

line_experiment = dict(color = df_bs_10['Constraints'], colorscale = [[0,'#1f77b4'], [1,'#ff7f0e']])

fig = go.Figure(data=go.Parcoords(line = line_experiment,
                                  dimensions = dimensions_metrics+dimensions_hyperparameters
                                  ))

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    title_text='Test Batch Size 10',
    title_x=0.5)

fig.show()

In [ ]:
dimensions_hyperparameters = list([
                                   dict(range = [8, 64], label = 'batchsize', values = df_bs_10['batchsize']),
                                   dict(range = [3, 9], label = 'conv1_kernel_size', values = df_bs_10['conv1_kernel_size']),
                                   dict(range = [16, 64], label = 'conv1_out_channels', values = df_bs_10['conv1_out_channels']),
                                   dict(range = [3, 9], label = 'conv2_kernel_size', values = df_bs_10['conv2_kernel_size']),
                                   dict(range = [32, 128], label = 'conv2_out_channels', values = df_bs_10['conv2_out_channels']),
                                   dict(range = [3, 9], label = 'conv3_kernel_size', values = df_bs_10['conv3_kernel_size']),
                                   dict(range = [64, 256], label = 'conv3_out_channels', values = df_bs_10['conv3_out_channels']),
                                   dict(range = [0.01, 0.5], label = 'dropout_probability', values = df_bs_10['dropout_probability']),
                                   dict(range = [3, 20], label = 'epochs', values = df_bs_10['epochs']),
                                   dict(range = [256, 1024], label = 'fc1_out_features', values = df_bs_10['fc1_out_features']),
                                   dict(range = [64, 512]	, label = 'fc2_out_features', values = df_bs_10['fc2_out_features']),
                                   dict(range = [0.00001, 0.5], label = 'learning_rate', values = df_bs_10['learning_rate']),
                                   dict(range = [2, 3], label = 'pool_kernel_size', values = df_bs_10['pool_kernel_size']),
                                   ])

dimensions_metrics = list([
                           dict(range = range_accuracy, label = 'accuracy', values = df_bs_10['accuracy']),
                           dict(range = range_peak_memory, label = 'peak_memory', values = df_bs_10['peak_memory']),
                           dict(range = range_storage_memory, label = 'storage_memory', values = df_bs_10['storage_memory']),
                           dict(range = range_inference_time, label = 'inference_time', values = df_bs_10['inference_time']),
                           ])

line_experiment = dict(color = df_bs_10['Constraints'], colorscale = [[0,'#1f77b4'], [1,'#ff7f0e']])

fig = go.Figure(data=go.Parcoords(line = line_experiment,
                                  dimensions = dimensions_metrics#+dimensions_hyperparameters,
                                  ))

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    title_text='Test Batch Size 10',
    title_x=0.5,
    width=500,
    height=500,)

fig.show()

In [ ]:
dimensions_hyperparameters = list([
                                   dict(range = [8, 64], label = 'batchsize', values = df_bs_20['batchsize']),
                                   dict(range = [3, 9], label = 'conv1_kernel_size', values = df_bs_20['conv1_kernel_size']),
                                   dict(range = [16, 64], label = 'conv1_out_channels', values = df_bs_20['conv1_out_channels']),
                                   dict(range = [3, 9], label = 'conv2_kernel_size', values = df_bs_20['conv2_kernel_size']),
                                   dict(range = [32, 128], label = 'conv2_out_channels', values = df_bs_20['conv2_out_channels']),
                                   dict(range = [3, 9], label = 'conv3_kernel_size', values = df_bs_20['conv3_kernel_size']),
                                   dict(range = [64, 256], label = 'conv3_out_channels', values = df_bs_20['conv3_out_channels']),
                                   dict(range = [0.01, 0.5], label = 'dropout_probability', values = df_bs_20['dropout_probability']),
                                   dict(range = [3, 20], label = 'epochs', values = df_bs_20['epochs']),
                                   dict(range = [256, 1024], label = 'fc1_out_features', values = df_bs_20['fc1_out_features']),
                                   dict(range = [64, 512]	, label = 'fc2_out_features', values = df_bs_20['fc2_out_features']),
                                   dict(range = [0.00001, 0.5], label = 'learning_rate', values = df_bs_20['learning_rate']),
                                   dict(range = [2, 3], label = 'pool_kernel_size', values = df_bs_20['pool_kernel_size']),
                                   ])

dimensions_metrics = list([
                           dict(range = range_accuracy, label = 'accuracy', values = df_bs_20['accuracy']),
                           dict(range = range_peak_memory, label = 'peak_memory', values = df_bs_20['peak_memory']),
                           dict(range = range_storage_memory, label = 'storage_memory', values = df_bs_20['storage_memory']),
                           dict(range = range_inference_time, label = 'inference_time', values = df_bs_20['inference_time']),
                           ])

line_experiment = dict(color = df_bs_20['Constraints'], colorscale = [[0,'#1f77b4'], [1,'#ff7f0e']])

fig = go.Figure(data=go.Parcoords(line = line_experiment,
                                  dimensions = dimensions_metrics+dimensions_hyperparameters
                                  ))

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    title_text='Test Batch Size 20',
    title_x=0.5)

fig.show()

In [ ]:
dimensions_hyperparameters = list([
                                   dict(range = [8, 64], label = 'batchsize', values = df_bs_20['batchsize']),
                                   dict(range = [3, 9], label = 'conv1_kernel_size', values = df_bs_20['conv1_kernel_size']),
                                   dict(range = [16, 64], label = 'conv1_out_channels', values = df_bs_20['conv1_out_channels']),
                                   dict(range = [3, 9], label = 'conv2_kernel_size', values = df_bs_20['conv2_kernel_size']),
                                   dict(range = [32, 128], label = 'conv2_out_channels', values = df_bs_20['conv2_out_channels']),
                                   dict(range = [3, 9], label = 'conv3_kernel_size', values = df_bs_20['conv3_kernel_size']),
                                   dict(range = [64, 256], label = 'conv3_out_channels', values = df_bs_20['conv3_out_channels']),
                                   dict(range = [0.01, 0.5], label = 'dropout_probability', values = df_bs_20['dropout_probability']),
                                   dict(range = [3, 20], label = 'epochs', values = df_bs_20['epochs']),
                                   dict(range = [256, 1024], label = 'fc1_out_features', values = df_bs_20['fc1_out_features']),
                                   dict(range = [64, 512]	, label = 'fc2_out_features', values = df_bs_20['fc2_out_features']),
                                   dict(range = [0.00001, 0.5], label = 'learning_rate', values = df_bs_20['learning_rate']),
                                   dict(range = [2, 3], label = 'pool_kernel_size', values = df_bs_20['pool_kernel_size']),
                                   ])

dimensions_metrics = list([
                           dict(range = range_accuracy, label = 'accuracy', values = df_bs_20['accuracy']),
                           dict(range = range_peak_memory, label = 'peak_memory', values = df_bs_20['peak_memory']),
                           dict(range = range_storage_memory, label = 'storage_memory', values = df_bs_20['storage_memory']),
                           dict(range = range_inference_time, label = 'inference_time', values = df_bs_20['inference_time']),
                           ])

line_experiment = dict(color = df_bs_20['Constraints'], colorscale = [[0,'#1f77b4'], [1,'#ff7f0e']])

fig = go.Figure(data=go.Parcoords(line = line_experiment,
                                  dimensions = dimensions_metrics#+dimensions_hyperparameters
                                  ))

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    title_text='Test Batch Size 20',
    title_x=0.5,
    width=500,
    height=500,)

fig.show()